<a href="https://colab.research.google.com/github/Jcc329/Jessica_DATA606/blob/main/Notebooks/2.Steam_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 2: Data Cleaning

This Notebook aims to clean and prep the data for analysis

### Data cleaning

The final dataset contains several columns with many values (subdictionaries).
To handle these, I will identify all columns containing desired data, remove unneccessary columns and use the apply function to convert the multidimentional columns into their own dataframes that can be appened back onto the original dataframe. 

#### Text Cleaning

Any text data will undergo additional cleaning to prepare it for analysis, including converting the text to lowercase, removing symbols and punctuation, and generally tidying the data.

This concludes the goals of this notebook. A cleaned dataset will be save and the next stage of EDA will occur in the next notebook in this series. 


In [1]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
#import raw data

GameData = pd.read_csv('RawSteamGameData2.csv',  engine='python')

In [3]:
GameData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18424 entries, 0 to 18423
Data columns (total 63 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                18424 non-null  int64  
 1   type                      18424 non-null  object 
 2   name                      18408 non-null  object 
 3   steam_appid               18424 non-null  int64  
 4   required_age              18424 non-null  int64  
 5   is_free                   18424 non-null  bool   
 6   detailed_description      17611 non-null  object 
 7   about_the_game            17611 non-null  object 
 8   short_description         18073 non-null  object 
 9   fullgame                  6776 non-null   object 
 10  header_image              18424 non-null  object 
 11  website                   10439 non-null  object 
 12  pc_requirements           18424 non-null  object 
 13  mac_requirements          18424 non-null  object 
 14  linux_

In [4]:
GameData.isnull().sum()

Unnamed: 0                      0
type                            0
name                           16
steam_appid                     0
required_age                    0
is_free                         0
detailed_description          813
about_the_game                813
short_description             351
fullgame                    11648
header_image                    0
website                      7985
pc_requirements                 0
mac_requirements                0
linux_requirements              0
developers                    851
publishers                      0
price_overview               4834
packages                     4531
package_groups                  0
platforms                       0
screenshots                   826
release_date                    0
support_info                    0
background                    821
content_descriptors             0
supported_languages           705
categories                    842
genres                       1189
movies        

In [5]:
#drop fields with a lot of mising data and fields that are unneccessary for the current analysis.
# Dropping 'drm_notice', 'alternate_appid','score_rank', 'ext_user_account_notice', 'demos', 'dlc', 'recommendations  high number of missing data
# Dropping 'legal_notice', 'header_image', 'website', 'pc_requirements', 'packages','mac_requirements', 'linux_requirements', 'screenshots', 'movies', due to irrelevance
#Dropping duplicated developer, publisher, reviews, price overview, languages
#Acheivements, dlc, and controller support are all covered in categories
#Only 250 games include metacritic scores. I will keep the field however, as I may decide to use it during exploratory analysis, or if I expand the number of games collected

GameData = GameData[['Unnamed: 0', 'type', 'name', 'steam_appid', 'required_age', 'is_free',
       'detailed_description', 'about_the_game', 'short_description',
       'supported_languages', 'developers', 'publishers', 'platforms',
       'categories', 'genres', 'release_date','content_descriptors', 'Review Score',
       'Review Score Description', 'Top Reviews by Upvotes', 'appid',
       'positive', 'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
       'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
       'ccu', 'genre', 'tags', 'metacritic'
       ]]

In [6]:
GameData.describe(include = 'all')

,Unnamed: 0,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,developers,publishers,platforms,categories,genres,release_date,content_descriptors,Review Score,Review Score Description,Top Reviews by Upvotes,appid,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,genre,tags,metacritic
count,1.842400e+04,18424,18408,1.842400e+04,18424.000000,18424,17611,17611,18073,17719,17573,18424,18424,17582,17235,18424,18424,18422.000000,18422,10190,1.842200e+04,1.842200e+04,18422.000000,18422.000000,18422,18422.000000,18422.000000,18422.000000,18422.000000,17007.000000,17007.000000,17007.000000,1.842200e+04,15920,18422,656
unique,NaN,10,16098,NaN,NaN,2,15274,15273,15563,2951,10041,8366,6,2511,1066,3511,1587,NaN,19,8833,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,8213,557
top,NaN,game,"Trainz 2019 DLC: Sebino Lake, Italy",NaN,NaN,False,About：<br />\r\nThe classic tank battle game r...,About：<br />\r\nThe classic tank battle game r...,"Pixel Puzzles Ultimate, redefining digital puz...",English,"['SmiteWorks USA, LLC']",[''],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': ''}","{'ids': [], 'notes': None}",NaN,No user reviews,6/10,NaN,NaN,NaN,NaN,"0 .. 20,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Action,[],"{'score': 87, 'url': 'https://www.metacritic.c..."
freq,NaN,11201,5,NaN,NaN,16596,18,18,20,4569,347,2469,12827,3073,893,254,15834,NaN,8229,29,NaN,NaN,NaN,NaN,15895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,882,7406,4
mean,9.584813e+05,NaN,NaN,1.014737e+06,0.293096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.669308,NaN,NaN,1.014816e+06,1.259372e+03,177.808436,0.065737,NaN,62.788459,3.639507,55.019542,3.683476,669.376316,692.958899,2.856412,1.433175e+03,NaN,NaN,NaN
std,5.073354e+05,NaN,NaN,4.981157e+05,2.199407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.897592,NaN,NaN,4.979958e+05,4.745422e+04,6523.135592,2.242255,NaN,580.320889,62.174160,513.002715,73.777249,1103.216223,1119.106874,12.309929,1.826654e+05,NaN,NaN,NaN
min,7.000000e+00,NaN,NaN,1.000000e+01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,1.000000e+01,0.000000e+00,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,NaN,NaN,NaN
25%,5.523950e+05,NaN,NaN,6.037275e+05,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,6.038100e+05,0.000000e+00,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,99.000000,99.000000,0.000000,0.000000e+00,NaN,NaN,NaN
50%,9.229250e+05,NaN,NaN,1.011725e+06,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,1.011725e+06,2.000000e+00,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,374.000000,399.000000,0.000000,0.000000e+00,NaN,NaN,NaN
75%,1.389835e+06,NaN,NaN,1.438870e+06,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,1.438820e+06,2.500000e+01,8.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,999.000000,999.000000,0.000000,0.000000e+00,NaN,NaN,NaN


In [7]:
GameData.head()

,Unnamed: 0,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,developers,publishers,platforms,categories,genres,release_date,content_descriptors,Review Score,Review Score Description,Top Reviews by Upvotes,appid,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,genre,tags,metacritic
0,1804560,music,ARKOS Soundtrack,1804560,0,False,It's the soundtrack for Arkos by kubikámi<br /...,It's the soundtrack for Arkos by kubikámi<br /...,It's the soundtrack for Arkos by kubikámi,NaN,['RetroSouls'],['RetroSouls'],"{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,"{'coming_soon': False, 'date': '10 Nov, 2021'}","{'ids': [], 'notes': None}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1676720,game,White Eyes,1676720,0,False,The universe implodes and interstellar nomads ...,The universe implodes and interstellar nomads ...,The universe implodes and interstellar nomads ...,English<strong>*</strong><br><strong>*</strong...,['Chocolate Ship Games'],['Chocolate Ship Games'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': '26 Jul, 2021'}","{'ids': [], 'notes': None}",0.0,1 user reviews,Don't normally like games that stretch my ref...,1676720.0,1.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,1499.0,1499.0,0.0,0.0,"Action, Casual, Indie","{'Action': 73, 'Casual': 67, 'Arcade': 45, ""Sh...",NaN
2,700820,game,TFM: The First Men,700820,0,False,"<h1>JOIN TFM DISCORD!</h1><p><a href=""https://...","TFM: The First Men is a strategy simulation, e...",A strategy simulation based on stories of the ...,"English<strong>*</strong>, French, Italian, Ge...",['Gathering Tree'],['Gathering Tree'],"{'windows': True, 'mac': True, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '23', 'description': 'Indie'}, {'id': ...","{'coming_soon': True, 'date': 'Spring 2022'}","{'ids': [], 'notes': None}",0.0,No user reviews,NaN,700820.0,0.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Indie, Simulation, Strategy, Early Access",[],NaN
3,1423248,dlc,My Neighborhood Arcade: RC Targets - Dronez Unit,1423248,0,False,Pilot an RC drone through a target course In R...,Pilot an RC drone through a target course In R...,Pilot an RC drone through a target course In R...,English<strong>*</strong><br><strong>*</strong...,['Layer Arcade'],['Simulated Entertainment'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': '26 Sep, 2020'}","{'ids': [], 'notes': None}",0.0,No user reviews,NaN,1423248.0,0.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,399.0,399.0,0.0,0.0,"Action, Casual, Indie, Racing",[],NaN
4,17095,dlc,My Neighborhood Arcade: RC Targets - Dronez Unit,1423248,0,False,Pilot an RC drone through a target course In R...,Pilot an RC drone through a target course In R...,Pilot an RC drone through a target course In R...,English<strong>*</strong><br><strong>*</strong...,['Layer Arcade'],['Simulated Entertainment'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': '26 Sep, 2020'}","{'ids': [], 'notes': None}",0.0,No user reviews,NaN,1423248.0,0.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,399.0,399.0,0.0,0.0,"Action, Casual, Indie, Racing",[],NaN


## A major issue with this data are the collumns that contain multiple datapoints. There are 3 formats this takes in  the data frame right now:
- a dictionary
- a list stored as a string
- a list of dictionaries stored as strings

I also need to convert the acheivements value into a binary (I don't care what the acheivements are, only whether or not it's a feature in the game). 

In [8]:
#Expand columns containing multiple datapoints
#https://stackoverflow.com/questions/38231591/split-explode-a-column-of-dictionaries-into-separate-columns-with-pandas

# I will start by handling dictionary lists stored as strings 
# create a function to convert the lists of dictionaries stored as strings to dictionaries for processing
# https://www.geeksforgeeks.org/python-convert-string-to-list-of-dictionaries/

def makelist(DictStrList):
    ''' 
    Takes list of dictionaries stored as a string and converts to a list of dictionaries
    '''
    try:    
        if len(DictStrList.split(', '))==2:
            x = eval(DictStrList)
        else:
            x = list(eval(DictStrList))
    except: 
        x = DictStrList
        # print(DictStrList)
    return x

In [9]:
# Handle categories
#desired output 1 column for each category type with a value indicating whether or not the game includes that value
#Start by finding each category name:
GameData2 = GameData
GameData2['categories'] = GameData2['categories'].apply(makelist)
GameData2 = GameData2.explode('categories')
categories = pd.concat([GameData2.drop(['categories'], axis=1), GameData2['categories'].apply(pd.Series)], axis=1)
uniquecats = list(categories['description'].unique()) #create list of unique categories
uniquecats #remove categories not in english

[nan,
 'Single-player',
 'Steam Achievements',
 'Steam Workshop',
 'Steam Cloud',
 'Includes level editor',
 'Downloadable Content',
 'Multi-player',
 'Shared/Split Screen',
 'Full controller support',
 'Steam Trading Cards',
 'Remote Play Together',
 'Partial Controller Support',
 'Steam Leaderboards',
 'Captions available',
 'Stats',
 'PvP',
 'Online PvP',
 'Co-op',
 'Online Co-op',
 'Native Steam Controller Support',
 'MMO',
 'In-App Purchases',
 'Shared/Split Screen PvP',
 'Shared/Split Screen Co-op',
 'Cross-Platform Multiplayer',
 'LAN PvP',
 'LAN Co-op',
 'Valve Anti-Cheat enabled',
 'Remote Play on TV',
 'VR Support',
 'Additional High-Quality Audio',
 'Remote Play on Phone',
 'Remote Play on Tablet',
 'Commentary available',
 'Steam Turn Notifications',
 'Game demo',
 'SteamVR Collectibles',
 '单人',
 'Includes Source SDK']

In [10]:
#remove categories not in english
remove = ['多人', '合作','部分支持控制器', '包含关卡编辑器', 'Steam 排行榜', '玩家对战', '线上玩家对战', '同屏/分屏玩家对战', '同屏/分屏',
       '统计数据', '远程同乐', '在线合作', 'Steam 创意工坊', '单人', 'Steam 成就', '完全支持控制器', 'Steam 集换式卡牌','Steam 云']
catlist = [cat for cat in uniquecats if cat not in remove]
#remove nan value
catlist = [cat for cat in catlist if pd.isnull(cat) == False]
catlist

['Single-player',
 'Steam Achievements',
 'Steam Workshop',
 'Steam Cloud',
 'Includes level editor',
 'Downloadable Content',
 'Multi-player',
 'Shared/Split Screen',
 'Full controller support',
 'Steam Trading Cards',
 'Remote Play Together',
 'Partial Controller Support',
 'Steam Leaderboards',
 'Captions available',
 'Stats',
 'PvP',
 'Online PvP',
 'Co-op',
 'Online Co-op',
 'Native Steam Controller Support',
 'MMO',
 'In-App Purchases',
 'Shared/Split Screen PvP',
 'Shared/Split Screen Co-op',
 'Cross-Platform Multiplayer',
 'LAN PvP',
 'LAN Co-op',
 'Valve Anti-Cheat enabled',
 'Remote Play on TV',
 'VR Support',
 'Additional High-Quality Audio',
 'Remote Play on Phone',
 'Remote Play on Tablet',
 'Commentary available',
 'Steam Turn Notifications',
 'Game demo',
 'SteamVR Collectibles',
 'Includes Source SDK']

In [11]:
# GameData['categories']

In [12]:
#Next iterate through list and create lists for each category to append
def valueoffered(dfCol, valuelist):
    ''' 
    Takes a data frame column in string  format and list of strings and iterates through the column looking for those strings
    returns a dataframe with a column for each value.
    '''
    valueDict = {}
    for value in valuelist: #start a list in a dictionary for each category or other variable
        valueDict[value] = []
    print(valueDict)
    for row in dfCol:
        try:
            for value in valuelist:
                if value in row: #if the game has that category tag, add a 1 to the category column, else add a 0
                    valueDict[value].append(1)
                else:
                    valueDict[value].append(0)
        except: #handle Nans
            for value in valuelist:
                valueDict[value].append(0)
    return pd.DataFrame(valueDict)

CategoryDF = valueoffered(GameData['categories'].astype(str), catlist)
CategoryDF

{'Single-player': [], 'Steam Achievements': [], 'Steam Workshop': [], 'Steam Cloud': [], 'Includes level editor': [], 'Downloadable Content': [], 'Multi-player': [], 'Shared/Split Screen': [], 'Full controller support': [], 'Steam Trading Cards': [], 'Remote Play Together': [], 'Partial Controller Support': [], 'Steam Leaderboards': [], 'Captions available': [], 'Stats': [], 'PvP': [], 'Online PvP': [], 'Co-op': [], 'Online Co-op': [], 'Native Steam Controller Support': [], 'MMO': [], 'In-App Purchases': [], 'Shared/Split Screen PvP': [], 'Shared/Split Screen Co-op': [], 'Cross-Platform Multiplayer': [], 'LAN PvP': [], 'LAN Co-op': [], 'Valve Anti-Cheat enabled': [], 'Remote Play on TV': [], 'VR Support': [], 'Additional High-Quality Audio': [], 'Remote Play on Phone': [], 'Remote Play on Tablet': [], 'Commentary available': [], 'Steam Turn Notifications': [], 'Game demo': [], 'SteamVR Collectibles': [], 'Includes Source SDK': []}


,Single-player,Steam Achievements,Steam Workshop,Steam Cloud,Includes level editor,Downloadable Content,Multi-player,Shared/Split Screen,Full controller support,Steam Trading Cards,Remote Play Together,Partial Controller Support,Steam Leaderboards,Captions available,Stats,PvP,Online PvP,Co-op,Online Co-op,Native Steam Controller Support,MMO,In-App Purchases,Shared/Split Screen PvP,Shared/Split Screen Co-op,Cross-Platform Multiplayer,LAN PvP,LAN Co-op,Valve Anti-Cheat enabled,Remote Play on TV,VR Support,Additional High-Quality Audio,Remote Play on Phone,Remote Play on Tablet,Commentary available,Steam Turn Notifications,Game demo,SteamVR Collectibles,Includes Source SDK
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18419,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18420,1,0,0,0,1,1,1,0,0,0,0,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18421,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18422,1,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
#add category df to GameData
GameData = pd.concat([GameData, CategoryDF], axis=1)

In [14]:
GameData.head().T

,0,1,2,3,4
Unnamed: 0,1804560,1676720,700820,1423248,17095
type,music,game,game,dlc,dlc
name,ARKOS Soundtrack,White Eyes,TFM: The First Men,My Neighborhood Arcade: RC Targets - Dronez Unit,My Neighborhood Arcade: RC Targets - Dronez Unit
steam_appid,1804560,1676720,700820,1423248,1423248
required_age,0,0,0,0,0
is_free,False,False,False,False,False
detailed_description,It's the soundtrack for Arkos by kubikámi<br /...,The universe implodes and interstellar nomads ...,"<h1>JOIN TFM DISCORD!</h1><p><a href=""https://...",Pilot an RC drone through a target course In R...,Pilot an RC drone through a target course In R...
about_the_game,It's the soundtrack for Arkos by kubikámi<br /...,The universe implodes and interstellar nomads ...,"TFM: The First Men is a strategy simulation, e...",Pilot an RC drone through a target course In R...,Pilot an RC drone through a target course In R...
short_description,It's the soundtrack for Arkos by kubikámi,The universe implodes and interstellar nomads ...,A strategy simulation based on stories of the ...,Pilot an RC drone through a target course In R...,Pilot an RC drone through a target course In R...
supported_languages,NaN,English<strong>*</strong><br><strong>*</strong...,"English<strong>*</strong>, French, Italian, Ge...",English<strong>*</strong><br><strong>*</strong...,English<strong>*</strong><br><strong>*</strong...


In [15]:
#handle genres
GameData2 = GameData
GameData2['genres'] = GameData2['genres'].apply(makelist)
GameData2 = GameData2.explode('genres')
genres =  GameData2['genres'].apply(pd.Series)
uniquegenres = list(genres['description'].unique()) #create list of unique genres
uniquegenres #remove genres not in english


[nan,
 'Action',
 'Casual',
 'Indie',
 'Simulation',
 'Strategy',
 'Early Access',
 'Racing',
 'RPG',
 'Adventure',
 'Education',
 'Game Development',
 'Free to Play',
 'Massively Multiplayer',
 'Design & Illustration',
 'Web Publishing',
 'Sexual Content',
 'Violent',
 'Gore',
 'Sports',
 'Movie',
 'Animation & Modeling',
 'Tutorial',
 'Audio Production',
 'Software Training',
 'Utilities',
 'Video Production',
 'Photo Editing',
 'Short',
 'Nudity',
 'Documentary',
 'Episodic',
 '360 Video',
 '冒险',
 '休闲',
 '独立',
 'Accounting']

In [16]:
#handle genres
#remove genres not in english
remove = ['抢先体验','策略','角色扮演','模拟', '免费开玩','休闲', '动作', '独立','冒险',]
genlist = [gen for gen in uniquegenres if gen not in remove]
#remove nan value
genlist = [gen for gen in genlist if pd.isnull(gen) == False]
genlist

['Action',
 'Casual',
 'Indie',
 'Simulation',
 'Strategy',
 'Early Access',
 'Racing',
 'RPG',
 'Adventure',
 'Education',
 'Game Development',
 'Free to Play',
 'Massively Multiplayer',
 'Design & Illustration',
 'Web Publishing',
 'Sexual Content',
 'Violent',
 'Gore',
 'Sports',
 'Movie',
 'Animation & Modeling',
 'Tutorial',
 'Audio Production',
 'Software Training',
 'Utilities',
 'Video Production',
 'Photo Editing',
 'Short',
 'Nudity',
 'Documentary',
 'Episodic',
 '360 Video',
 'Accounting']

In [17]:
GenreDF = valueoffered(GameData['genres'].astype(str),genlist)
GenreDF.head()

{'Action': [], 'Casual': [], 'Indie': [], 'Simulation': [], 'Strategy': [], 'Early Access': [], 'Racing': [], 'RPG': [], 'Adventure': [], 'Education': [], 'Game Development': [], 'Free to Play': [], 'Massively Multiplayer': [], 'Design & Illustration': [], 'Web Publishing': [], 'Sexual Content': [], 'Violent': [], 'Gore': [], 'Sports': [], 'Movie': [], 'Animation & Modeling': [], 'Tutorial': [], 'Audio Production': [], 'Software Training': [], 'Utilities': [], 'Video Production': [], 'Photo Editing': [], 'Short': [], 'Nudity': [], 'Documentary': [], 'Episodic': [], '360 Video': [], 'Accounting': []}


,Action,Casual,Indie,Simulation,Strategy,Early Access,Racing,RPG,Adventure,Education,Game Development,Free to Play,Massively Multiplayer,Design & Illustration,Web Publishing,Sexual Content,Violent,Gore,Sports,Movie,Animation & Modeling,Tutorial,Audio Production,Software Training,Utilities,Video Production,Photo Editing,Short,Nudity,Documentary,Episodic,360 Video,Accounting
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
#add Genre df to GameData
GameData = pd.concat([GameData, GenreDF], axis = 1)

## Next I will handle fields that are stored as dictionaries by turning the column of dictionaries into a dataframe and appending to the original dataset.

In [19]:
# convert nans to empty dict
def handleNaN(col):
    newlist = []
    for item in col:
        if str(item) == 'nan':
            newlist.append({})
        else:
            newlist.append(item)
    return newlist

GameData['metacritic'] = handleNaN(GameData['metacritic'])
GameData['platforms'] = handleNaN(GameData['platforms'])
GameData['release_date'] = handleNaN(GameData['release_date'])
GameData['content_descriptors'] = handleNaN(GameData['content_descriptors'])


In [20]:
# https://stackoverflow.com/questions/38231591/split-explode-a-column-of-dictionaries-into-separate-columns-with-pandas

platforms = pd.json_normalize(GameData['platforms'].astype(str).apply(eval))
release = pd.json_normalize(GameData['release_date'].astype(str).apply(eval))
content = pd.json_normalize(GameData['content_descriptors'].astype(str).apply(eval))
metacritic = pd.json_normalize(GameData['metacritic'].astype(str).apply(eval))
#Rename columns
release = release.rename({'date':'Release Date'})
content = content.rename({'notes': 'Content Notes'})
content = content.drop(columns = 'ids')
metacritic = metacritic.rename({'score':'Metacritic Score'})
metacritic = metacritic.drop(columns = 'url')

In [21]:
#Add all new data frames back into the original

GameData = pd.concat([GameData, platforms, release, content, metacritic], axis = 1)

In [22]:
GameData.head()

,Unnamed: 0,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,developers,publishers,platforms,categories,genres,release_date,content_descriptors,Review Score,Review Score Description,Top Reviews by Upvotes,appid,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,genre,tags,metacritic,Single-player,Steam Achievements,Steam Workshop,Steam Cloud,Includes level editor,Downloadable Content,Multi-player,Shared/Split Screen,Full controller support,Steam Trading Cards,Remote Play Together,Partial Controller Support,Steam Leaderboards,Captions available,...,Remote Play on TV,VR Support,Additional High-Quality Audio,Remote Play on Phone,Remote Play on Tablet,Commentary available,Steam Turn Notifications,Game demo,SteamVR Collectibles,Includes Source SDK,Action,Casual,Indie,Simulation,Strategy,Early Access,Racing,RPG,Adventure,Education,Game Development,Free to Play,Massively Multiplayer,Design & Illustration,Web Publishing,Sexual Content,Violent,Gore,Sports,Movie,Animation & Modeling,Tutorial,Audio Production,Software Training,Utilities,Video Production,Photo Editing,Short,Nudity,Documentary,Episodic,360 Video,Accounting,windows,mac,linux,coming_soon,date,notes,score
0,1804560,music,ARKOS Soundtrack,1804560,0,False,It's the soundtrack for Arkos by kubikámi<br /...,It's the soundtrack for Arkos by kubikámi<br /...,It's the soundtrack for Arkos by kubikámi,NaN,['RetroSouls'],['RetroSouls'],"{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,"{'coming_soon': False, 'date': '10 Nov, 2021'}","{'ids': [], 'notes': None}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,False,False,False,"10 Nov, 2021",None,NaN
1,1676720,game,White Eyes,1676720,0,False,The universe implodes and interstellar nomads ...,The universe implodes and interstellar nomads ...,The universe implodes and interstellar nomads ...,English<strong>*</strong><br><strong>*</strong...,['Chocolate Ship Games'],['Chocolate Ship Games'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': '26 Jul, 2021'}","{'ids': [], 'notes': None}",0.0,1 user reviews,Don't normally like games that stretch my ref...,1676720.0,1.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,1499.0,1499.0,0.0,0.0,"Action, Casual, Indie","{'Action': 73, 'Casual': 67, 'Arcade': 45, ""Sh...",{},1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,False,False,False,"26 Jul, 2021",None,NaN
2,700820,game,TFM: The First Men,700820,0,False,"<h1>JOIN TFM DISCORD!</h1><p><a href=""https://...","TFM: The First Men is a strategy simulation, e...",A strategy simulation based on stories of the ...,"English<strong>*</strong>, French, Italian, Ge...",['Gathering Tree'],['Gathering Tree'],"{'windows': True, 'mac': True, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '23', 'description': 'Indie'}, {'id': ...","{'coming_soon': True, 'date': 'Spring 2022'}","{'ids': [], 'notes': None}",0.0,No user reviews,NaN,700820.0,0.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Indie, Simulation, Strategy, Early Access",[],{},1,1,1,1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,True,False,True,Spring 2022,None,NaN
3,1423248,dlc,My Neighborhood Arcade: RC Targets - Dronez Unit,1423248,0,False,Pilot an RC drone through a target course In R...,Pilot an RC drone through a target course In R...,Pilot an RC drone through a target course In R...,English<strong>*</strong><br><strong>*</strong...,['Layer Arcade'],['Simulated Entertainment'],"{'windows': True, 'mac': Fa

#Text Data Cleaning

Target fields:
- detailed description
- short description
- about the game
- top reviews by upvote
- tags (note, I am treating tags as a text field because there are so many possible tags a game can have)

Initial Cleaning goals:
- Send to lowercase
- remove punctuation and symbols
- remove stopwords

In [23]:
import string
import re
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [24]:
# https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string
# https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python

# preprocessing for tags 
GameData['tags'] = GameData['tags'].str.strip('[]')
textfields = ['detailed_description', 'about_the_game', 'short_description', 'Top Reviews by Upvotes', 'tags']

def CleanText(DF, ColList, stop_words):
    '''
    Input: Dataframe containing text data, list of columns to clean, stopwords to remove
    Output: Dataframe containing cleaned text data
    '''
    CleanedCol = {}
    for col in ColList:
        stringlist = []
        for line in DF[col]:
            #remove html tags and text in brackets
            cleanstring = re.sub(re.compile('<.*?>') , '', str(line))
            cleanstring = re.sub(re.compile('\[.*?]') , ' ', cleanstring)
            cleanstring = re.sub(re.compile('\n') , ' ', cleanstring)
            #Remove stop words and sent to lowercase
            cleanstring = " ".join([word.lower() for word in str(cleanstring).split() if word.lower() not in stop_words])
            #Remove emojis and icons
            icons = re.compile("["u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002500-\U00002BEF"  # chinese char
                        "]+", re.UNICODE)
            cleanstring = re.sub(icons, '', cleanstring)
            #Remove punctuation
            for punctuation in string.punctuation: 
                cleanstring = cleanstring.replace(punctuation, '')
            
            stringlist.append(cleanstring)
        CleanedCol.update({col:stringlist})
    return pd.DataFrame(CleanedCol)

CleanData = CleanText(GameData, textfields, stop_words=stop_words)

In [25]:
CleanData


,detailed_description,about_the_game,short_description,Top Reviews by Upvotes,tags
0,soundtrack arkos kubikámi pure midi music rend...,soundtrack arkos kubikámi pure midi music rend...,soundtrack arkos kubikámi,nan,nan
1,universe implodes interstellar nomads race get...,universe implodes interstellar nomads race get...,universe implodes interstellar nomads race get...,normally like games stretch reflexes far game ...,action 73 casual 67 arcade 45 shoot em up 40 2...
2,join tfm discordwishlist nowabout gametfm firs...,tfm first men strategy simulation enriched cha...,strategy simulation based stories first men in...,nan,
3,pilot rc drone target course rc targets dronez...,pilot rc drone target course rc targets dronez...,pilot rc drone target course rc targets dronez...,nan,
4,pilot rc drone target course rc targets dronez...,pilot rc drone target course rc targets dronez...,pilot rc drone target course rc targets dronez...,nan,
...,...,...,...,...,...
18419,far would go save best friendfor gomo answer q...,far would go save best friendfor gomo answer q...,far would go save best friendfor gomo answer q...,short amazingly fun gomo tells short story kid...,adventure 99 indie 78 point click 43 puzzle 2...
18420,il2 sturmovik great battles series combat flig...,il2 sturmovik great battles series combat flig...,mobile aaa gun allows protect armored column s...,milk truck goodgreat download like tank crew p...,
18421,kingdom’s future rests light three beacons lit...,kingdom’s future rests light three beacons lit...,kingdom’s future rests light three beacons lit...,ive purchased versions dark flight thinking on...,adventure 23 casual 22
18422,gain items accessories inspired vtuber group q...,gain items accessories inspired vtuber group q...,gain items accessories inspired vtuber group q...,nan,


In [26]:
Clean_GameData = pd.concat([GameData, CleanData], axis = 1)

In [27]:
from google.colab import files
Clean_GameData.to_csv('CleanSteamGameData.csv') 
files.download('CleanSteamGameData.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>